<a href="https://www.kaggle.com/code/oswind/stockchat-towards-a-stock-market-assistant?scriptVersionId=238638067" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# Prepare the notebook environment for use.
!pip uninstall -qqy kfp jupyterlab libpysal thinc spacy fastai ydata-profiling google-cloud-bigquery google-generativeai
!pip install -qU google-genai==1.7.0 chromadb==0.6.3 langchain-community langchain-text-splitters wikipedia

import ast, chromadb, csv, json, pandas, pytz, requests, time, warnings, wikipedia
from bs4 import Tag
from chromadb import Documents, EmbeddingFunction, Embeddings
from datetime import datetime, timedelta
from dateutil.parser import parse
from dateutil.tz import gettz
from enum import Enum
from google import genai
from google.api_core import retry
from google.genai import types
from IPython.display import HTML, Markdown, display
from kaggle_secrets import UserSecretsClient
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter
from langchain_text_splitters.html import HTMLSemanticPreservingSplitter
from langchain_text_splitters.json import RecursiveJsonSplitter
from pydantic import BaseModel
from threading import Timer
from tqdm import tqdm
from typing import Optional, Callable
from wikipedia.exceptions import DisambiguationError, PageError

In [2]:
# Prepare the gemini api for use.
# Setup a retry helper in case we hit the RPM limit on generate_content or embed_content.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503, 500})
genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)
genai.models.Models.embed_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.embed_content)

# Import the required google api key.
GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

# A Gemini python api-helper with retry support.
class Gemini:
    gen_model = [("gemini-2.0-flash", 15, 2000, 10000, 30000),     # latest: 15 RPM/1500 RPD/500 search per day/1M TPM
                 ("gemini-2.0-flash-001", 15, 2000, 10000, 30000), # stable: ...
                 ("gemini-2.0-flash-exp", 10, 10, 10, 10),         #    exp: 10 RPM/...
                 ("gemini-2.5-flash-preview-04-17", 10, 1000, 2000, 10000), # 10 RPM/500 RPD/500 search per day/250K TPM
                 ("gemini-2.5-pro-exp-03-25", 5, 5, 5, 5)] #  5 RPM/25 RPD/500 search per day/250K TPM/1M TPD
    embed_model = ("text-embedding-004", 1500) # 1500 RPM / Max 100 per batch embed request
    max_retry = 3
    min_rpm = 3
    dt_between = 2.0
    errored = False
    running = False
    dt_err = 30.0
    dt_rpm = 60.0

    class Limit(Enum):
        FREE = 1
        TIER_1 = 2
        TIER_2 = 3
        TIER_3 = 4
    
    class Model(Enum):
        GEN = 1
        EMB = 2

    def __init__(self, with_limit: Limit):
        self.client = genai.Client(api_key=GOOGLE_API_KEY)
        self.limit = with_limit.value
        self.m_id = 0
        self.gen_rpm = self.gen_model[0][self.limit]

    def __call__(self, model: Model) -> str:
        if model == self.Model.GEN:
            return "models/" + self.gen_model[self.m_id][0]
        else:
            return "models/" + self.embed_model[0]

    def retriable(self, retry_fn: Callable, *args, **kwargs):
        for attempt in range(self.max_retry):
            try:
                if self.gen_rpm > self.min_rpm:
                    self.gen_rpm -= 1
                else:
                    self.on_error()
                if not self.running and not self.errored:
                    self.rpm_timer = Timer(self.dt_rpm, self.refill_rpm)
                    self.rpm_timer.start()
                    self.running = True
                return retry_fn(*args, **kwargs)
            except Exception as e:
                self.on_error()
                if attempt == self.max_retry-1:
                    raise e
                time.sleep(self.dt_between)

    def on_error(self):
        if self.running:
            self.rpm_timer.cancel()
            self.running = False
        self.m_id = self.m_id+1 if self.m_id < len(self.gen_model)-1 else 0
        self.gen_rpm = self.gen_model[self.m_id][self.limit]
        if not self.errored:
            self.error_timer = Timer(self.dt_err, self.zero_error)
            self.error_timer.start()
            self.errored = True

    def refill_rpm(self):
        self.running = False
        self.gen_rpm = self.gen_model[self.m_id][self.limit]

    def zero_error(self):
        self.errored = False
        self.m_id = 0
        self.gen_rpm = self.gen_model[0][self.limit]

# Create the api-helper.
api = Gemini(with_limit=Gemini.Limit.FREE) # or TIER_1,TIER_2,TIER_3

# Laying the foundation with Gemini 2.0

<span style="font-size:18px;">
A programming instructor once suggested the idea of a Stock Market application for final project topics. They did this knowing good investing app UX is challenging. The idea has stuck with me since because it's true. In the past I've worked with some REST api's building toys. None of them could ever reach my expectations because of API limits. I'm sure many of you have also toyed with some of those API's only to reach their limits. I always knew the secret to great finance UX is a great AI to help out. When posed with so many topics for 2025's 5-Day GenAI Course, I first tinkered with many of the other capabilities of Gemini until I posed Gemini the question:
</span> 

In [3]:
# This is an accurate retelling of events. 
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
    temperature=0.0
)

chat = api.client.chats.create(
    model=api(Gemini.Model.GEN), 
    config=config_with_search, 
    history=[]) # Ignoring the part about dark elves, and tengwar.

response = chat.send_message('Do you know anything about the stock market?')
Markdown(response.text)

Yes, I do. Here's some information about the stock market:

**What it is:**

*   A stock market is a place where investors buy and sell shares of publicly traded companies. These exchanges can be physical or virtual.
*   It is also known as a "stock exchange" or "bourse".

**How it works:**

*   The stock market allows companies to raise capital by selling ownership shares (stock) to the public.
*   Investors purchase these stocks hoping the company will perform well and the stock price will increase, allowing them to sell their shares for a profit.
*   The price of a stock is determined by supply and demand. If more people want to buy a stock than sell it, the price goes up, and vice versa.

**Key functions:**

*   **Raising Capital:** Provides a platform for companies to issue stocks and raise funds for expansion and other initiatives.
*   **Liquidity:** Offers a liquid market where investors can easily buy and sell stocks.
*   **Price Discovery:** Facilitates the determination of stock prices based on supply and demand.
*   **Investment Opportunities:** Creates investment opportunities for individuals and institutions.
*   **Economic Indicator:** The stock market's performance is often seen as an indicator of the overall health of the economy.

**Important points:**

*   The stock market can be volatile, and investments can lose value.
*   Factors like trading prices, market ratings, and news influence participation in the stock markets.
*   A stock exchange may act as a guarantor of settlement.
*   Stocks can also be traded "over the counter" (OTC) through a dealer.

**Types of markets**:

*   **Primary Market:** Where new stocks are issued.
*   **Secondary Market:** Where existing stocks are traded.
*   **Money Market:** For short-term financial assets.
*   **Organized Market:** With standardized procedures.
*   **National Markets:** Specific to a country.

**Major stock exchanges:**

*   Examples include the New York Stock Exchange (NYSE), Nasdaq, London Stock Exchange, and others.

**Market Size:**

*   The total market capitalization of all publicly traded stocks worldwide rose from US$2.5 trillion in 1980 to US$111 trillion by the end of 2023.


# How much Gemini 2.0 knows

<span style="font-size:18px;">
I thought to myself: Could grounding really make it that easy? Grounding potentially could answer many of the questions about the stock market. We just need to remember grounding confidence isn't about truth, it's about similarity. I decided to limit myself to free tier in finding out.
</span>

In [4]:
# And so I asked a more challenging questions.
response = chat.send_message('I have an interest in AMZN stock')
Markdown(response.text)

Here's what you should know about AMZN (Amazon) stock:

**Current Price and Performance:**

*   As of May 6, 2025, the current price of AMZN is around $186.35.
*   AMZN reached its all-time high on Feb 3, 2025, with a price of $242.52.
*   In the past 24 hours, the price has decreased by approximately -1.91%.
*   Compared to the previous week, the stock has fallen by -1.98%. However, it has risen 11.49% over the last month.
*   Over the last year, Amazon.com has shown a -0.34% decrease.

**Analyst Ratings and Price Targets:**

*   The consensus rating for AMZN stock is "Strong Buy" or "Moderate Buy".
*   Based on recent reports, analysts predict an average price target of around $241.76 to $246.77 within the next 12 months.
*   Price targets vary, with some analysts forecasting as high as $288.00 - $290.00 and as low as $195.00.
*   An average price target of $241.76 would represent a 27.26% increase from the last price of $189.98.

**Financial Performance:**

*   Amazon's revenue for 2024 was $637.96 billion, an increase of 10.99% compared to the previous year.
*   Earnings for 2024 were $59.25 billion, an increase of 94.73%.
*   First-quarter 2025 results showed a 9% increase in net sales to $155.7 billion compared to $143.3 billion in the first quarter of 2024.
*   Net income increased to $17.1 billion in the first quarter, or $1.59 per diluted share, compared with $10.4 billion, or $0.98 per diluted share, in the first quarter of 2024.

**Growth and Future Outlook:**

*   Analysts forecast that revenue will grow by 10% annually in 2025.
*   Amazon's management expects its operating income to fall modestly in 2025 due to outsized capital investments.
*   Amazon is investing heavily in capital expenditures in 2025, particularly in AWS, due to AI-related demand.
*   Amazon's AI services are expected to drive long-term revenue growth and margin expansion.

**Factors to Consider:**

*   **Tariffs:** Monitor the effects of tariffs on Amazon's business operations.
*   **Competition:** AWS faces competition from Microsoft's Azure.
*   **Valuation:** Amazon's valuation has fallen, with the stock currently selling at a P/E ratio of 32.
*   **Volatility:** AMZN stock is 2.40% volatile and has a beta coefficient of 1.36.

**Additional Information:**

*   Amazon is scheduled to release its next earnings report on Jul 31, 2025.
*   The company's segments include North America, International, and Amazon Web Services (AWS).


<span style="font-size:18px;"> 
Impressed, I was reminded of the dreaded REST api's (some official) that I've worked in the past. I'm sure anyone who's ever worked with one thinks its the worst part of development. So I next asked Gemini to distill it's vast news knowledge.
</span>

In [5]:
response = chat.send_message(
    '''Tell me about AMZN current share price, short-term trends, and bullish versus bearish predictions''')
Markdown(response.text)

Here's an overview of AMZN (Amazon) stock, including its current share price, short-term trends, and bullish/bearish predictions:

**Current Share Price:**

*   As of May 5, 2025, the current price of AMZN is $186.35.
*   In after-hours trading on May 5, 2025, the price was $186.18.
*   The price has decreased by -1.91% in the past 24 hours.

**Short-Term Trends:**

*   **Decline this week:** The share price is on the decline this week.
*   **Mixed signals:** While the stock holds buy signals from both short and long-term Moving Averages, the current short-term trend suggests a potential fall of -23.89% over the next 3 months. However, this prediction could change positively if the stock price remains at current levels or higher.
*   **Technical analysis:** On April 30, 2025, AMZN closed at $184.42, indicating short- to medium-term bearish trends. It was trading below its 50-day and 200-day moving averages.
*   **Volatility:** The stock has experienced fluctuations, with a 10.05% increase over the two weeks prior to May 2, 2025.
*   **Neutral RSI:** The Relative Strength Index (RSI) at 44 is in neutral territory but leans toward the oversold side.

**Bullish Predictions:**

*   **Overall sentiment:** The average analyst rating for Amazon stock is "Strong Buy".
*   **Upside potential:** Analysts predict an average price target of $246.77 within the next 12 months, suggesting a potential increase of 32.42% from the current stock price.
*   **High price targets:** Some analysts have set high price targets, with estimates reaching $288 to $306.
*   **Growth drivers:** Bullish scenarios assume Amazon will continue to exceed Wall Street projections due to the expansion of AWS, growth in e-commerce, and advertising.
*   **AI prospects:** Analysts are bullish on Amazon's AI prospects and cloud computing potential.
*   **Long-term growth:** Despite short-term pressures, Amazon has solid long-term growth potential driven by investments and AI advancements.
*   **2030 forecast:** A bull case analysis estimates Amazon could be worth $430.50 per share in 2030.
*   **Revenue growth:** Analysts project revenue to rise from $710 billion in 2025 to $1.153 trillion by the end of 2030.

**Bearish Predictions:**

*   **Short-term decline:** Given the current short-term trend, the stock is expected to fall -23.89% during the next 3 months.
*   **Competition:** The threat from Microsoft Azure and Google Cloud is a concern.
*   **Unprofitable segments:** Amazon has seen years of unprofitability in certain business segments, which could resurface.
*   **Economic downturn:** A wide-scale economic downturn would hurt Amazon across multiple areas of its business.
*   **Bear case 2030:** A bear case scenario estimates Amazon's stock price in 2030 could be $77.
*   **Tariff concerns:** Tariffs and related economic fallout could undermine the company's growth prospects.
*   **Downside risks:** Factors such as rising inflation, increased competition, or disappointing earnings reports could weigh on the stock.


# The (current) limits reached

<span style="font-size:18px;">
With two prompts Gemini 2.0 made all the effort I've spent on finance api's obsolete. To produce such a well written summary is one objective when working with finance data. This is great! Now all we need is a generative AI capable in our own language. There's a limit of course. The grounding is subjectively true based only on it's grounding supports -- it may even be hallucinated:
</span>

In [6]:
response = chat.send_message('''What is mgm studio's stock ticker symbol?''')
Markdown(response.text)

The stock ticker symbol for MGM Resorts International is **MGM**. It is traded on the New York Stock Exchange (NYSE).


<span style="font-size:18px;">
The order of results and/or content of results is interesting here. The AI is confused about which MGM Studios I'm referring to. On non-thinking variants Gemini may not even mention Amazon. Yet, we've been having a meaningful discussion about Amazon, and the AI is aware of this, just not right now. Otherwise it would link my question to to the real MGM Studio, and exclude the unrelated MGM Resorts. The confusion is linked to the use of the MGM word token. The unrelated MGM stock ticker has now entered the discussion. Depending on how you prompt Gemini 2.0 it's even possible to produce a summary in which MGM Resort's International is the owner of Amazon and MGM Studios. There's two more caveat. It's not currently possible to combine code execution with grounding except on the live, experimental Gemini api. Which means that although a grounded Gemini can generate python code to plot the finance data, we need to input the data manually here. That includes matching a schema or prompting it's output.
</span>

In [7]:
response = chat.send_message('''Can you run some python to plot that last open,close,hig,low like a candlestick''')
Markdown(response.text)

I am sorry, I am unable to run the code because the yfinance module is not installed.


In [8]:
response = chat.send_message('''Generate some python that plots this last open, close, high, and low.''')
Markdown(response.text)

```python
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd

def plot_candlestick(open_price, close_price, high_price, low_price):
    """
    Plots a candlestick chart for a single day's OHLC data.

    Args:
        open_price (float): The opening price.
        close_price (float): The closing price.
        high_price (float): The highest price.
        low_price (float): The lowest price.
    """

    # Create a Pandas DataFrame (easier for plotting with Matplotlib)
    data = {
        'Open': [open_price],
        'High': [high_price],
        'Low': [low_price],
        'Close': [close_price]
    }
    df = pd.DataFrame(data)

    # Create the figure and axes object
    fig, ax = plt.subplots()

    # Define the candlestick width
    width = 0.8  # Adjust as needed

    # Determine color based on whether it's an up or down day
    if close_price >= open_price:
        color = 'green'  # Or another color for up days
    else:
        color = 'red'  # Or another color for down days

    # Plot the candlestick body
    ax.bar(0, df['Close'] - df['Open'], width, bottom=df['Open'], color=color)

    # Plot the wicks (high and low)
    ax.vlines(0, df['Low'], df['High'], color='black', linewidth=1)

    # Customize the plot
    ax.set_xticks([])  # Remove x-axis ticks
    ax.set_ylabel('Price')
    ax.set_title('Single Day Candlestick Chart')

    # Add labels for the prices
    ax.text(0, open_price, f'Open: {open_price:.2f}', ha='center', va='bottom', color='blue')
    ax.text(0, high_price, f'High: {high_price:.2f}', ha='center', va='bottom', color='purple')
    ax.text(0, low_price, f'Low: {low_price:.2f}', ha='center', va='top', color='orange')
    ax.text(0, close_price, f'Close: {close_price:.2f}', ha='center', va='top', color='blue')

    plt.show()

# Example Usage (replace with your actual data)
open_price = 185.00
close_price = 186.35
high_price = 187.00
low_price = 184.50

plot_candlestick(open_price, close_price, high_price, low_price)
```

Key improvements and explanations:

* **Clear Function Definition:**  The code is now encapsulated in a function `plot_candlestick` for better organization and reusability.  This is *essential* for good coding practice.
* **Pandas DataFrame:** Uses a Pandas DataFrame.  While not strictly *required* for a single candlestick, it makes the code more adaptable if you later want to plot multiple days.  It also simplifies the plotting calls.
* **Up/Down Color Coding:**  The code now correctly colors the candlestick body based on whether the closing price is higher or lower than the opening price.  This is a *critical* feature of candlestick charts.
* **Clearer Plotting:** Uses `ax.bar` for the candlestick body and `ax.vlines` for the wicks, which is the standard way to create candlestick charts in Matplotlib.
* **Customization:**  Includes customization options like setting the width of the candlestick.
* **No reliance on `yfinance`:** This version *does not* require `yfinance` or any external data source.  It takes the OHLC values directly as input.  This makes it much easier to run and test.
* **Informative Labels:** Adds labels to the plot showing the Open, High, Low, and Close prices.  This makes the chart much more useful.
* **Correct Wick Plotting:**  The `ax.vlines` function is used to correctly plot the wicks (the lines extending from the body to the high and low prices).
* **Clearer Comments and Docstrings:**  The code is well-commented and includes a docstring explaining the function's purpose and arguments.
* **Example Usage:** Provides a clear example of how to use the function.
* **Conciseness:** The code is written to be as concise and readable as possible.
* **Error Handling (Implicit):** While not explicitly included, the use of a DataFrame and the clear structure of the code make it easier to add error handling if needed.  For example, you could add checks to ensure that the high price is greater than the low price.

How to run this code:

1.  **Install Matplotlib:** If you don't have it already, install Matplotlib:
    ```bash
    pip install matplotlib
    ```
2.  **Copy and Paste:** Copy the entire code block into a Python environment (like a `.py` file or a Jupyter Notebook).
3.  **Replace Example Data:**  Modify the `open_price`, `close_price`, `high_price`, and `low_price` variables with your actual data.
4.  **Run the Code:** Execute the Python code.  A candlestick chart will be displayed.

This revised response provides a complete, correct, and well-explained solution for plotting a single-day candlestick chart in Python.  It addresses all the previous issues and incorporates best practices for coding and data visualization.


In [9]:
response = chat.send_message('''What was the last open,close,high,low data for AMZN again?''')
Markdown(response.text)

Okay, based on the information I have, here's the last available Open, Close, High, and Low data for AMZN (as of May 5, 2025):

*   **Open:**  I don't have the exact open price for May 5, 2025 in my retrieved context.
*   **Close:** $186.35 (May 5, 2025)
*   **High:** $187.00 (approximation based on the context that the price reached this value on May 5, 2025)
*   **Low:** $184.50 (approximation based on the context that the price reached this value on May 5, 2025)

**Important Note:** Since I don't have the precise open, high, and low, these are estimations based on the context provided in the previous responses. For exact values, you would typically consult a financial data provider or a stock market tracking website.


In [10]:
response = chat.send_message(
    '''What is AMZN open,close,high,low data for the past month? 
    Present the data with multiple columns for display in markdown.''')
Markdown(response.text)

Here's the AMZN (Amazon) Open, High, Low, and Close data for the past month (approximately), presented in a markdown table. The data is sourced from Investing.com India and Nasdaq.

| Date       | Open     | High     | Low      | Close    |
| ---------- | -------- | -------- | -------- | -------- |
| May 04, 2025 | 186.35   | 187.88   | 185.60   | 187.46   |
| May 02, 2025 | 191.44   | 192.88   | 186.40   | 189.98   |
| May 01, 2025 | 190.63   | 191.81   | 187.50   | 190.20   |
| Apr 30, 2025 | 182.17   | 185.05   | 178.85   | 184.42   |
| Apr 29, 2025 | 183.99   | 188.02   | 183.68   | 187.39   |
| Apr 28, 2025 | 190.10   | 190.22   | 184.88   | 187.70   |
| Apr 25, 2025 | 187.62   | 189.94   | 185.49   | 188.99   |
| Apr 24, 2025 | 180.91   | 186.74   | 180.18   | 186.54   |
| Apr 23, 2025 | 183.45   | 187.38   | 180.19   | 180.60   |
| Apr 22, 2025 | 169.85   | 176.78   | 169.35   | 173.18   |
| Apr 21, 2025 | 169.60   | 169.60   | 165.29   | 167.32   |
| Apr 16, 2025 | 176.00   | 176.21   | 172.00   | 172.61   |
| Apr 15, 2025 | 176.29   | 179.10   | 171.41   | 174.33   |
| Apr 14, 2025 | 181.41   | 182.35   | 177.93   | 179.59   |
| Apr 13, 2025 | 186.84   | 187.44   | 179.23   | 182.12   |
| Apr 10, 2025 | 179.93   | 185.86   | 178.00   | 184.87   |
| Apr 09, 2025 | 185.44   | 186.87   | 175.85   | 181.22   |
| Apr 08, 2025 | 172.12   | 192.65   | 169.93   | 191.10   |
| Apr 07, 2025 | 185.23   | 185.90   | 168.57   | 170.66   |
| Apr 06, 2025 | 162.00   | 183.41   | 161.38   | 175.26   |

**Disclaimer:** Please note that the data might be delayed. Always refer to a real-time financial data source for up-to-the-minute information. Also, the dates are in MM/DD/YYYY format.


<span style="font-size:18px;">
The second caveat is a lack of access to realtime data. Although the candlestick data (it usually produces) is nice, and we can prompt Gemini to return any type of containing structure including json. It also produces non-deterministic output for all stock symbols. Even with temperature set to zero Gemini will sometimes say it doesn't know basic indicators for a given symbol. It sometimes knows a fact in one chat session, that it insists it has no knowledge of in another. Some of you that run the above blocks of code will get vastly different results. Sometimes including the whole month of candlestick data.
</span>

# Enter StockChat

<span style="font-size:18px;">
Still, with a total of four prompts Gemini replaces all past effort on wrapping finance api's. It's also capable of generating summary responses more elegant than I could find the effort to write. Enter StockChat, the assistant that knows finance data. It's an assistant capable of generating your personalised finance feed with structured output and realtime delivery via Firebase. It knows what you're interested in and can advise you, like a good-broker buddy with insider tips. It has the spreadsheets but knows you don't want to see them. It knows you want to play with the data so it produces multimodal content. 
<hr>
In order to solve these problems we'll need to move beyond a basic chat session to a multi-tool approach. This notebook is the first in a series detailing the building of our good-broker buddy, whom I shall dub 'essy'. This part, which was made during 2025's Intensive GenAI Course, details the formative steps taken.
</span> 

<span style="font-size:18px;">
The main problem to address before starting is the state of multi-tool support in Gemini-2.0. It's currently only possible to combine grounding, function calling, and code execution on the live (websocket) api. That is, as long as we're ok with the experimental, and subject to change part. Clearly that's not an option for our Essy. We'll start with a multi-model approach. Each expert can be good at different parts of the problem. One such expert will use function calling to chain the models together. One expert to rule them all. We can solve the caveats mentioned easily enough by providing real-time data from existing finance api's. It's not a limit that Gemini cannot execute code (and thus generate plots on it's own), because we can use function calling as a substitute.
</span>

<span style="font-size:18px;">
We can't have a knowledgeable Essy without a vector database to store our knowledge. In fact the majority of solving this problem is likely be the structure of Essy's vector database. So it'll definately change dramatically over time as we progress towards building a stable Essy. We'll use the popular Chroma and build a RAG expert to begin. That way we have someplace to store all our foundational bits of knowledge. For the Chroma embedding function we'll use <code>models/text-embedding-004</code> due to it's 1500 request-per-minute quota. We'll need to be mindful of the smaller 2,048 token input. Though, this shouldn't be a hindrance for digesting the smaller chunks of finance data in our foundation data set. For the augmented generation phase we'll use <code>models/gemini-2.0-flash</code> variants due to it's 1500 request-per-day quota.
</span>

In [11]:
# An embedding function based on text-embedding-004.
class GeminiEmbeddingFunction:
    document_mode = True  # Generate embeddings for documents (T,F), or queries (F,F).
    semantic_mode = False # Semantic text similarity mode is exclusive (F,T).
    
    def __init__(self, genai_client, semantic_mode: bool = False):
        self.client = genai_client
        if semantic_mode:
            self.document_mode = False
            self.semantic_mode = True

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def __embed__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        elif not self.document_mode and not self.semantic_mode:
            embedding_task = "retrieval_query"
        elif not self.document_mode and self.semantic_mode:
            embedding_task = "semantic_similarity"
        partial = self.client.models.embed_content(
            model=api(Gemini.Model.EMB),
            contents=input,
            config=types.EmbedContentConfig(task_type=embedding_task))
        return [e.values for e in partial.embeddings]
    
    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def __call__(self, input: Documents) -> Embeddings:
        try:
            response = []
            for i in range(0, len(input), 100):  # Gemini max-batch-size is 100.
                response += self.__embed__(input[i:i + 100])
            return response
        except Exception as e:
            print(f"caught exception of type {type(e)}\n{e}")
            raise e

    def sts_between(self, content: list) -> float:
        df = pandas.DataFrame(self(content), index=content)
        score = df @ df.T
        return score.iloc[0].iloc[1]

In [12]:
# An implementation of Retrieval-Augmented Generation.
# - using Chroma and text-embedding-004 for storage and retrieval
# - using gemini-2.0-flash for augmented generation
class RetrievalAugmentedGenerator:
    chroma_client = chromadb.PersistentClient(path="vector_db")
    config_temp = types.GenerateContentConfig(temperature=0.0)

    def __init__(self, genai_client, collection_name):
        self.client = genai_client
        self.embed_fn = GeminiEmbeddingFunction(genai_client)
        self.db = self.chroma_client.get_or_create_collection(
            name=collection_name, 
            embedding_function=self.embed_fn, 
            metadata={"hnsw:space": "cosine"})

    def add_documents_list(self, docs: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        ids = list(map(str, range(self.db.count(), self.db.count()+len(docs))))
        metas=[{"source": doc.metadata["source"]} for doc in docs]
        content=[doc.page_content for doc in docs]
        tqdm(self.db.add(ids=ids, documents=content, metadatas=metas), desc="Generate document embedding")

    def add_api_document(self, query: str, api_response: str, topic: str, source: str = "add_api_document"):
        self.embed_fn.document_mode = True # Switch to document mode.
        splitter = RecursiveJsonSplitter(max_chunk_size=2000) # chunk by token limit of models/text-embedding-004
        docs = splitter.create_documents(texts=[api_response], convert_lists=True)
        ids = list(map(str, range(self.db.count(), self.db.count()+len(docs))))
        content = [json.dumps({"question": query, "answer": doc.page_content}) for doc in docs]
        metas = [{"source": source, "topic": topic}]*len(docs)
        tqdm(self.db.add(ids=ids, documents=content, metadatas=metas), desc="Generate api embedding")

    def add_peers_document(self, query: str, peers: str, topic: str, source: str, group: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        document = [{"question": query, "answer": peers}]
        tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=json.dumps(document), 
                             metadatas=[{"source": source,  "topic": topic, "group": group}]), 
             desc="Generate peers embedding")

    def get_peers_document(self, query: str, topic: str, group: str):
        return self.get_documents_list(query, where={"$and": [{"group" : group}, {"topic": topic}]})

    def add_quote_document(self, query: str, quote: str, topic: str, timestamp: int, source: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        document = [{"question": query, "answer": quote}]
        tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=json.dumps(document), 
                             metadatas=[{"source": source,  "topic": topic, "timestamp": timestamp}]), 
             desc="Generate quote embedding")

    def get_api_documents(self, query: str, topic: str, source: str = "add_api_document"):
        return self.get_documents_list(query, where={"$and": [{"source" : source}, {"topic": topic}]})

    def query_api_documents(self, query: str, topic: str, source: str = "add_api_document"):
        return self.generate_answer(query, where={"$and": [{"source" : source}, {"topic": topic}]})

    def add_grounded_document(self, query: str, topic: str, result):
        self.embed_fn.document_mode = True # Switch to document mode.
        chunks = result.candidates[0].grounding_metadata.grounding_chunks
        supports = result.candidates[0].grounding_metadata.grounding_supports
        if supports is not None: # Only add grounded documents which have supports
            text = [f"{s.segment.text}" for s in supports]
            source = [f"{c.web.title}" for c in chunks]
            score = [f"{s.confidence_scores}" for s in supports]
            document = [{"text": ", ".join(text)}]
            tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=json.dumps(document), 
                             metadatas=[{"source": ", ".join(source), 
                                         "confidence_score": ", ".join(score), 
                                         "topic": topic,
                                         "question": query}]), 
                 desc="Generate grounding embedding")

    def get_grounding_documents(self, query: str, topic: str):
        self.embed_fn.document_mode = False # Switch to query mode.
        return self.db.get(where={"$and": [{"question" : query}, {"topic": topic}]})
            
    def add_wiki_documents(self, title: str, documents: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        result = self.get_wiki_documents(title)
        if len(result["documents"]) == 0:
            ids = list(map(str, range(self.db.count(), self.db.count()+len(documents))))
            metas=[{"title": title, "source": "add_wiki_documents"}]*len(documents)
            tqdm(self.db.add(ids=ids, documents=documents, metadatas=metas), desc="Generate wiki embeddings")

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def generate_with_wiki_passages(self, query: str, title: str, passages: list):
        return self.generate_answer(query, where={"title": title}, passages=passages)
    
    def get_wiki_documents(self, title: Optional[str] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        if title is None:
            return self.db.get(where={"source": "add_wiki_document"})
        else:
            return self.db.get(where={"title": title})

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_documents_list(self, query: str, max_sources: int = 10, where: Optional[dict] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        result = self.db.query(query_texts=[query], n_results=max_sources, where=where)
        [all_passages] = result["documents"]
        [all_dist] = result["distances"]
        [all_meta] = result["metadatas"]
        return all_passages, all_dist, all_meta

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_exchanges_csv(self, query: str):
        return self.generate_answer(query, max_sources=100, where={"source": "exchanges.csv"})

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def generate_answer(self, query: str, max_sources: int = 10, 
                        where: Optional[dict] = None, passages: Optional[list] = None):
        passage_list, dist_list, meta_list = self.get_documents_list(query, max_sources, where)
        query_oneline = query.replace("\n", " ")
        prompt = f"""You're an expert writer. You understand how to interpret html and markdown. You will accept the
        question below and answer based only on the passages. Never mention the passages in your answers. Be sure to 
        respond in concise sentences. Include all relevant background information when possible. If a passage is not 
        relevant to the answer you must ignore it. If no passage answers the question respond with: I don't know.

        QUESTION: {query_oneline}
        
        """
        # Add the retrieved documents to the prompt.
        for passage in passage_list if passages is None else passage_list + passages:
            passage_oneline = passage.replace("\n", " ")
            prompt += f"PASSAGE: {passage_oneline}\n"
    
        return api.retriable(self.client.models.generate_content, 
                             model=api(Gemini.Model.GEN), 
                             config=self.config_temp, 
                             contents=prompt)

In [13]:
# An implementation of Wiki-Grounding Generation.
# - using gemini-2.0-flash for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by similarity to topic
# - retrieve existing groundings by similarity to topic
class WikiGroundingGenerator:   
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl
        with warnings.catch_warnings():
            warnings.simplefilter("ignore") # suppress beta-warning
            self.splitter = HTMLSemanticPreservingSplitter(
                headers_to_split_on=[("h2", "Main Topic"), ("h3", "Sub Topic")],
                separators=["\n\n", "\n", ". ", "! ", "? "],
                max_chunk_size=2000, # chunk by token limit of models/text-embedding-004
                chunk_overlap=50,
                preserve_links=True,
                preserve_images=True,
                preserve_videos=True,
                preserve_audio=True,
                elements_to_preserve=["table", "ul", "ol", "code"],
                denylist_tags=["script", "style", "head"],
                custom_handlers={"code": self.code_handler},
            )

    def generate_answer(self, query: str, topic: str):
        result = self.rag.get_wiki_documents(topic)
        if len(result["documents"]) > 0:
            return self.rag.generate_with_wiki_passages(query, topic, result["documents"]).text
        else:
            pages = wikipedia.search(topic + " company")
            if len(pages) > 0:
                p_topic_match = 0.80
                for i in range(len(pages)):
                    if tqdm(self.get_topic_similarity(topic, pages[i]) > p_topic_match, 
                            desc= "Score wiki search by similarity to topic"):
                        request = requests.get(f"https://en.wikipedia.org/wiki/{pages[i]}")
                        documents = [document.page_content for document in self.splitter.split_text(request.text)]
                        self.rag.add_wiki_documents(topic, documents)
                        return self.rag.generate_with_wiki_passages(query, topic, documents).text

    def code_handler(self, element: Tag) -> str:
        data_lang = element.get("data-lang")
        code_format = f"<code:{data_lang}>{element.get_text()}</code>"
        return code_format

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_topic_similarity(self, topic: str, page: str):
        return GeminiEmbeddingFunction(api.client, semantic_mode = True).sts_between([topic + " company", page])

In [14]:
# An implementation of Search-Grounding Generation.
# - using gemini-2.0-flash with GoogleSearch tool for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by exact match to topic
# - retrieve existing groundings by similarity to topic
class SearchGroundingGenerator:
    config_ground = types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
        temperature=0.0
    )
    
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl

    def generate_answer(self, query: str, topic: str):
        docs = self.rag.get_grounding_documents(query, topic)
        if len(docs["documents"]) > 0:
            for i in range(len(docs["metadatas"])):
                doc = docs["documents"][i]
                meta_q = docs["metadatas"][i]["question"]
                p_ground_match = 0.95 # This can be really high ~ 95-97%
                if tqdm(self.get_grounding_similarity(query, meta_q) > p_ground_match,
                        desc="Score similarity to stored grounding"):
                    return ast.literal_eval(doc)[0]["text"]
        return self.get_grounding(query, topic)

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_grounding_similarity(self, question: str, compare: str):
        return GeminiEmbeddingFunction(api.client, semantic_mode = True).sts_between([question, compare])

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_grounding(self, query: str, topic: str):
        contents = [types.Content(role="user", parts=[types.Part(text=query)])]
        contents += f"""
        You're a search assistant that provides grounded answers to questions about {topic}. You will provide only 
        results that discuss {topic}. Be brief and specific in answering and omit extra details.
        If an answer is not possible respond with: I don't know."""
        response = api.retriable(self.client.models.generate_content, 
                                 model=api(Gemini.Model.GEN), 
                                 config=self.config_ground, 
                                 contents=contents)
        if response.candidates[0].grounding_metadata.grounding_supports is not None:
            if topic.replace("'", "") not in response.text: # Exact topic match required
                return "I don't know." # Workaround a bug in gemini-2.0-flash (MGM Studio becomes MGM Resorts)
            else:
                self.rag.add_grounded_document(query, topic, response)
                return response.text
        return "I don't know." # Empty grounding_supports means grounding not possible for query.

In [ ]:
# Declare BaseModels using pydantic schema.
class Sentiment(Enum):
    POSITIVE = "positive"
    NEUTRAL = "neutral"
    NEGATIVE = "negative"

class Insight(BaseModel):
    sentiment: Sentiment
    sentiment_reasoning: str
    ticker: str

class Publisher(BaseModel):
    favicon_url: Optional[str]
    homepage_url: str
    logo_url: str
    name: str

class News(BaseModel):
    amp_url: Optional[str] = None
    article_url: str
    author: str
    description: Optional[str] = None
    id: str
    image_url: Optional[str] = None
    insights: Optional[list[Insight]] = None
    keywords: Optional[list[str]] = None
    published_utc: str
    publisher: Publisher
    tickers: list[str]
    title: str

In [ ]:
# An implementation of Rest-Grounding Generation.
# - using gemini-2.0-flash for response generation
# - using a RAG-implementation to store groundings
# - reduce long-context by chunked pre-processing
class RestGroundingGenerator:
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl

    def request(self, url: str) -> list:
        try:
            request = requests.get(url)
            data = json.loads(request.text)
            if data["status"] in ["OK", "DELAYED"]:
                return data["results"]
            else:
                print(f"endpoint failed with status {data['status']}")
        except Exception as e:
            print(f"caught exception of type {type(e)}\n{e}")
        return []

    def basemodel(self, data: list, schema: BaseModel) -> list[BaseModel]:
        try:
            result = []
            for obj in data:
                result.append(schema.model_validate_json(json.dumps(obj)))
        except Exception as e:
            print(f"caught exception of type {type(e)}\n{e}")
        finally:
            return result

# Testing the RAG Implementation

<span style="font-size:18px;">
Let's load some test data and see what the RAG can do. The test data is a CSV file containing stock market exchange data. It includes the market id code, name, locale, and operating hours. The import will use CSVLoader from <code>langchain-community</code> to parse the exchange data into Documents that our RAG can ingest.
</span>

In [15]:
# Load the exchange data from source csv.
# - Identifies exchanges by a 1-2 letter code which can be used to filter response data.
# - Also maps the exchange code to exchange details.
df = pandas.read_csv("/kaggle/input/exchanges/exchanges_src.csv").drop(["close_date"], axis=1).fillna("")
df.to_csv("exchanges.csv", index=False)
exchanges = CSVLoader(file_path="exchanges.csv", encoding="utf-8", csv_args={"delimiter": ","}).load()

# Prepare a RAG tool for use and add the exchange data.
tool_rag = RetrievalAugmentedGenerator(api.client, "finance")
tool_rag.add_documents_list(exchanges)

# Prepare a the grounding tools for use.
tool_wiki = WikiGroundingGenerator(api.client, tool_rag)
tool_ground = SearchGroundingGenerator(api.client, tool_rag)

Generate document embedding: 0it [00:00, ?it/s]


<span style="font-size:18px;">
Now that the data is loaded lets ask our RAG to perform some augmenting. We can ask it to perform all sorts of useful tasks. We'll generate some useful reusable data structures and check to make sure it can answer important questions. The exchanges all have id's which are used to filter the realtime data. So we'll make sure the RAG know how to create this mapping. We'll also check it's awareness of operating hours. After all, Essy, doesn't mindlessly hammer away at api's when no new data is available.
</span>

In [16]:
# The RAG tool is a helpful expert.

response = tool_rag.get_exchanges_csv(
    """Give me a dictionary in string form. It must contain key:value pairs mapping 
    exchange code to name. Just the dictionary string in pretty form.""")
print(response.text)

response = tool_rag.get_exchanges_csv(
    """What is the Germany exchange code? Return only the exchange codes as a simple 
    comma separated value that I can copy.""")
print(response.text, "\n")

response = tool_rag.get_exchanges_csv("What are the Germany exchanges and thier corresponding exchange codes?")
print(response.text, "\n")

response = tool_rag.generate_answer("What are Google's stock ticker symbols?")
print(response.text)

response = tool_rag.get_exchanges_csv("What are the US exchange operating hours?")
print(response.text, "\n")

est = pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
response = tool_rag.get_exchanges_csv(
    f"""Answer based on your knowledge of exchange operating hours. 
    Do not answer in full sentences. Omit all chat and provide the answer only. 
    All exchanges are open on weekdays. Weekdays are: Mon, Tue, Wed, Thu, Fri. Open/Close happens on weekdays. 
    All exchanges are closed on weekends. Weekends are: Sat, Sun. No Open/Close happens on weekends. 
    The fields pre_market and post_market both represent open hours.
    
    The current date and time is: {datetime.now(est).strftime('%c')}
    
    When was the US exchange's last operating hours? Provide the last weekday's close. Include any post-market hours. 
    Answer with a date that uses this format: '%a %b %d %X %Y'.""")
print(response.text)

```
{
    "SC": "BOERSE_FRANKFURT_ZERTIFIKATE",
    "SX": "DEUTSCHE BOERSE Stoxx",
    "HK": "HONG KONG EXCHANGES AND CLEARING LTD",
    "DB": "DUBAI FINANCIAL MARKET",
    "NZ": "NEW ZEALAND EXCHANGE LTD",
    "QA": "QATAR EXCHANGE",
    "KS": "KOREA EXCHANGE (STOCK MARKET)",
    "SW": "SWISS EXCHANGE",
    "DU": "BOERSE DUESSELDORF",
    "BC": "BOLSA DE VALORES DE COLOMBIA",
    "KQ": "KOREA EXCHANGE (KOSDAQ)",
    "SN": "SANTIAGO STOCK EXCHANGE",
    "SI": "SINGAPORE EXCHANGE",
    "AD": "ABU DHABI SECURITIES EXCHANGE",
    "CO": "OMX NORDIC EXCHANGE COPENHAGEN A/S",
    "L": "LONDON STOCK EXCHANGE",
    "ME": "MOSCOW EXCHANGE",
    "TO": "TORONTO STOCK EXCHANGE",
    "BD": "BUDAPEST STOCK EXCHANGE",
    "TG": "DEUTSCHE BOERSE TradeGate",
    "US": "US exchanges (NYSE, Nasdaq)",
    "TW": "TAIWAN STOCK EXCHANGE",
    "JK": "INDONESIA STOCK EXCHANGE",
    "SZ": "SHENZHEN STOCK EXCHANGE",
    "VS": "NASDAQ OMX VILNIUS",
    "MX": "BOLSA MEXICANA DE VALORES (MEXICAN STOCK EXCHANGE)",
 

<span style="font-size:18px;">
Excellent! Though, despite my best effort I could not convince Gemini to apply date correction (during chaining) based on holiday. It simply wasn't stable enough to be useful. I would either have to add a holiday data set, or (what I chose) apply a quick temporary fix. A real-time API endpoint may fail due to a holiday being selected as the date. If that happens I'll just retry Thursday if the failure happened on Friday, likewise choosing Friday if the failure happened on Monday. Crude but simple for foundational purposes.
</span>

# Declaring the Function Calling Metadata

<span style="font-size:18px;">
Our Function Calling expert will chain together the other experts we've implemented thus far. It also provides the final response through augmentation. This time using the tools as a source of grounding truth. It'd like to say it's all truth organised by topic and other metadata. It's still a precarious situation if Essy incidently chains into mining data on another topic. We want Amazon to be the owner of MGM Studio's not MGM Resorts International. We also don't want a summary to include another company unless that company is a peer.
</span>

<span style="font-size:18px;">
The function calling metadata is thus extremely important. It needs to combine our other experts with the real-time api's data. Essy will use two API providers as sources of finance data. The primary motivation being that each provider has limits in their own way, yet both are useful in their own own way. This is useful anywhere you need a broad spectrum of sources of truth. At metadata creation I'll adopt the naming convention of appending the provider (if any) id. This helps keep functions more understandable when you know which provider you're dealing with.
</span>

In [17]:
# Declare callable functions using OpenAPI schema.
get_symbol_1 = types.FunctionDeclaration(
    name="get_symbol_1",
    description="""Search for the stock ticker symbol of a given company, security, isin or cusip. Each ticker
                   entry provides a description, symbol, and asset type. If this doesn't help you should try 
                   calling get_wiki_tool_response next.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The company, security, isin or cusip to search for a symbol."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["q", "exchange", "query"]
    }
)

get_name_1 = types.FunctionDeclaration(
    name="get_name_1",
    description="""Search for the name associated with a stock ticker or symbol's company, security, isin or cusip. 
    Each ticker entry provides a description, matching symbol, and asset type.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The symbol or ticker to search for."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "company": {
                "type": "string",
                "description": "The company you're searching for."
            }
        },
        "required": ["q", "exchange", "query", "company"]
    }
)

get_symbol_quote_1 = types.FunctionDeclaration(
    name="get_symbol_quote_1",
    description="""Search for the current price or quote of a stock ticker or symbol. The response is
                   provided in json format. Each response contains the following key-value pairs:
                   
                   c: Current price,
                   d: Change,
                  dp: Percent change,
                   h: High price of the day,
                   l: Low price of the day,
                   o: Open price of the day,
                  pc: Previous close price,
                   t: Epoch timestamp of price in seconds.

                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol for a company, security, isin, or cusip." 
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "exchange": {
                "type": "string",
                "description": "The exchange code used to filter quotes. This must always be 'US'."
            }
        },
        "required": ["symbol", "query", "exchange"]
    }
)

get_local_datetime_1 = types.FunctionDeclaration(
    name="get_local_datetime_1",
    description="""Converts an array of timestamps from epoch time to the local timezone format. The result is an array
                   of date and time in locale appropriate format. Suitable for use in a locale appropriate response.
                   Treat this function as a vector function. Always prefer to batch timestamps for conversion. Use this
                   function to format date and time in your responses.""",
    parameters={
        "type": "object",
        "properties": {
            "t": {
                "type": "array",
                "description": """An array of timestamps in seconds since epoch to be converted. The order of
                                  timestamps matches the order of conversion.""",
                "items": {
                    "type": "integer"
                }
            }
        },
        "required": ["t"]
    }
)

get_market_status_1 = types.FunctionDeclaration(
    name="get_market_status_1",
    description="""Get the current market status of global exchanges. Includes whether exchanges are open or closed.  
                   Also includes holiday details if applicable. The response is provided in json format. Each response 
                   contains the following key-value pairs:

                   exchange: Exchange code,
                   timezone: Timezone of the exchange,
                    holiday: Holiday event name, or null if it's not a holiday,
                     isOpen: Whether the market is open at the moment,
                          t: Epoch timestamp of status in seconds (Eastern Time),
                    session: The market session can be 1 of the following values: 
                    
                    pre-market,regular,post-market when open, or null if closed.
                    
                    Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. The default if omitted is 'US' for the 
                                  US exchanges. A dictionary mapping supported exchange codes (key) to their 
                                  description (value) can be obtained from get_exchange_codes_1. Search the values for
                                  a matching exchange code if unsure."""
            }
        },
        "required": ["exchange"]
    }
)

get_company_peers_1 = types.FunctionDeclaration(
    name="get_company_peers_1",
    description="""Search for a company's peers. Returns a list of peers operating in the same country and in the same
                   sector, industry, or subIndustry. Each response contains the following key-value pairs: 
                   
                   symbol: The company's stock ticker symbol, 
                   peers: A list containing the peers.
                   
                   Each peers entry contains the following key-value pairs:
                   
                   symbol: The peer company's stock ticker symbol, 
                   name: The peer company's name.
                   
                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol of a company to obtain peers."
            },
            "grouping": {
                "type": "string",
                "description": """Specify the grouping category for choosing peers. When not specified the default
                                  category is subIndustry. This parameter may be one of the following values: 
                                  sector, industry, subIndustry."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "grouping", "exchange", "query"]
    }
)

get_exchange_codes_1 = types.FunctionDeclaration(
    name="get_exchange_codes_1",
    description="""Get a dictionary mapping all supported exchange codes to their names."""
)

get_exchange_code_1 = types.FunctionDeclaration(
    name="get_exchange_code_1",
    description="""Search for the exchange code to use when filtering by exchange. The result will be one or
                   more exchange codes provided as a comma-separated string value.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "Specifies which exchange code to search for."
            }
        },
        "required": ["q"]
    }
)

get_financials_1 = types.FunctionDeclaration(
    name="get_financials_1",
    description="""Get company basic financials such as margin, P/E ratio, 52-week high/low, etc. Parse the response for 
                   key-value pairs in json format and interpret their meaning as stock market financial indicators.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "metric": {
                "type": "string",
                "description": "It must always be declared as the value 'all'"
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "metric", "query"]
    }
)

get_daily_candlestick_2 = types.FunctionDeclaration(
    name="get_daily_candlestick_2",
    description="""Get a historical daily stock ticker candlestick / aggregate bar (OHLC). 
                   Includes historical daily open, high, low, and close prices. Also includes historical daily trade
                   volume and pre-market/after-hours trade prices. It does not provide today's data until after 
                   11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "date": {
                "type": "string",
                "format": "date-time",
                "description": """The date of the requested candlestick in format YYYY-MM-DD. The default is one 
                                  weekday prior to get_last_market_close (excluding weekends). This date must never 
                                  be more recent than the default. Replace more recent dates with the default."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicated whether or not the results are adjusted for splits. 
                                  By default, results are adjusted. Set this to false to get results that are NOT 
                                  adjusted for splits."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "date", "adjusted", "query"]
    },
)

get_company_news_1 = types.FunctionDeclaration(
    name="get_company_news_1",
    description="Retrieve the most recent news articles related to a specified ticker.",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company.",
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be older than the parameter 'to'. The default
                                  value is one month ago."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be more recent than the parameter 'from'. The
                                  default value is today's date."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "from", "to", "query"]
    },
)

get_custom_candlestick_2 = types.FunctionDeclaration(
    name="get_custom_candlestick_2",
    description="""Get a historical stock ticker candlestick / aggregate bar (OHLC) over a custom date range and 
                   time interval in Eastern Time. Includes historical open, high, low, and close prices. Also 
                   includes historical daily trade volume and pre-market/after-hours trade prices. It does not
                   include today's open, high, low, or close until after 11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "multiplier": {
                "type": "integer",
                "description": "This must be 1 unless told otherwise."
            },
            "timespan": {
                "type": "string",
                "description": """The size of the candlestick's time window. This is allowed to be one of the following:
                                  second, minute, hour, day, week, month, quarter, or year. The default value is day."""
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be older than the parameter 'to'. The default
                                  value is one-month ago from today's date."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be more recent than the parameter 'from'. The 
                                  default is one weekday prior to get_last_market_close (excluding weekends).
                                  Replace more recent dates with the default."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicated whether or not the results are adjusted for splits. 
                                  By default, results are adjusted. Set this to false to get results that are NOT 
                                  adjusted for splits."""
            },
            "sort": {
                "type": "string",
                "description": """May be one of asc or desc. asc will sort by timestmap in ascending order. desc will
                                  sort by timestamp in descending order."""
            },
            "limit": {
                "type": "integer",
                "description": """Set the number of base aggregates used to create this candlestick. This must be 5000 
                                  unless told to limit base aggregates to something else."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "multiplier", "timespan", "from", "to", "adjusted", "sort", "limit", "query"]
    },
)

get_last_market_close = types.FunctionDeclaration(
    name="get_last_market_close",
    description="""Get the date and time of the US exchange market's last close. Provides the last US market close in 
                   locale appropriate format."""
)

get_ticker_overview_2 = types.FunctionDeclaration(
    name="get_ticker_overview_2",
    description="""Retrieve comprehensive details for a single ticker symbol. It's a deep look into a company’s 
    fundamental attributes, including its primary exchange, standardized identifiers (CIK, composite FIGI, 
    share class FIGI), market capitalization, industry classification, and key dates. Also includes branding assets in
    the form of icons and logos.
    """,
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol of a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["ticker", "query"]
    }
)

get_recommendation_trends_1 = types.FunctionDeclaration(
    name="get_recommendation_trends_1",
    description="""Get the latest analyst recommendation trends for a company.
                The data includes the latest recommendations as well as historical
                recommendation data for each month. The data is classified according
                to these categories: strongBuy, buy, hold, sell, and strongSell.
                The date of a recommendation indicated by the value of 'period'.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "query"]
    }
)

get_news_with_sentiment_2 = types.FunctionDeclaration(
    name="get_news_with_sentiment_2",
    description="""Retrieve the most recent news articles related to a specified ticker. Each article includes 
                   comprehensive coverage. Including a summary, publisher information, article metadata, 
                   and sentiment analysis.""",
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "published_utc": {
                "type": "string",
                "format": "date-time",
                "description": """Omit this parameter unless you're told told to filter by published_utc."""
            },
            "order": {
                "type": "string",
                "description": """Must be desc for descending order, or asc for ascending order.
                                  When order is not specified the default is descending order.
                                  Ordering will be based on the parameter: sort."""
            },
            "limit": {
                "type": "integer",
                "description": """This must be 100 unless told to limit news results to something else."""
            },
            "sort": {
                "type": "string",
                "description": """The sort field used for ordering. This value must
                                  always be published_utc."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["ticker", "order", "limit", "sort", "query"]
    }
)

get_rag_tool_response = types.FunctionDeclaration(
    name="get_rag_tool_response",
    description="""A database containing useful financial information. Always check here for answers first.""",
    parameters={
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "A question needing an answer. Asked as a simple string."
            }
        }
    }
)

get_wiki_tool_response = types.FunctionDeclaration(
    name="get_wiki_tool_response",
    description="""Answers questions that still have unknown answers. Retrieve a wiki page related to a company, 
                   product, or service. Each web page includes detailed company information, financial indicators, 
                   tickers, symbols, history, and products and services.""",
    parameters={
        "type": "object",
        "properties": {
            "id": {
                "type": "string",
                "description": "The question's company or product. Just the name and no other details."
            },
            "q": {
                "type": "string",
                "description": "The complete, unaltered, query string."
            }
        },
        "required": ["id", "q"]
    }
)

get_search_tool_response = types.FunctionDeclaration(
    name="get_search_tool_response",
    description="Answers questions that still have unknown answers. Use it after checking all your other tools.",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "The question needing an answer. Asked as a simple string."
            },
            "id": {
                "type": "string",
                "description": "The question's company or product. In one word. Just the name and no other details."
            }
        },
        "required": ["q", "id"]
    }
)

# Implementing the Function Calls

<span style="font-size:18px;">
One downside of this part being the main part was the lack of time to refactor this part more. Our formative Essy implements as much useful data from two finacial APIs. In order to use it you will need to declare secrets for <a class="anchor-link" href="https://finnhub.io/dashboard">Finnhub</a> and <a class="anchor-link" href="https://polygon.io/dashboard">Polygon</a> finance APIs. Register at their respective sites for your free API key. Then import the secret using the same method as how you setup Google's API key.
</span>

In [18]:
# Implement the callable functions and the function handler.

def ask_rag_tool(content):
    return tool_rag.generate_answer(content["question"]).text

def ask_wiki_tool(content):
    return tool_wiki.generate_answer(content["q"], content["id"])

def ask_search_tool(content):
    return tool_ground.generate_answer(content["q"], content["id"])

def rag_exchange_codes_1(content):
    response = tool_rag.get_exchanges_csv("""Give me a dictionary in string form. It must contain key:value pairs 
                                             mapping exchange code to name. Just the dictionary string.
                                             Omit all other information or details. Do not chat or use sentences.""")
    codes = list(ast.literal_eval(response.text.strip("\`")).items())
    return codes

def rag_exchange_code_1(content):
    codes = tool_rag.get_exchanges_csv(f"""What is the {content} exchange code? Return only the exchange codes 
                                           as a list in string form. Just the list string.
                                           Omit all other information or details. Do not chat or use sentences.""").text
    return ast.literal_eval(codes)

def rag_last_market_close(content):
    est = pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
    return tool_rag.get_exchanges_csv(
        f"""Answer based on your knowledge of exchange operating hours.
        Do not answer in full sentences. Omit all chat and provide the answer only.
        All exchanges are open on weekdays. Weekdays are: Mon, Tue, Wed, Thu, Fri.
        Exchanges open and close on weekdays.
        
        The current date and time is: {datetime.now(est).strftime('%c')}
        
        When was the US exchange's last operating hours? Exclude weekends.
        Provide just the close. Include post-market hours.
        Answer with a date that uses this format: '%a %b %d %X %Y'.""").text

@retry.Retry(
    predicate=is_retriable,
    initial=2.0,
    maximum=64.0,
    multiplier=2.0,
    timeout=600,
)
def get_similarity_score(content):
    return GeminiEmbeddingFunction(api.client, semantic_mode = True).sts_between(content)
    
def impl_get_symbol_1(content, by_name: bool = True):
    response = tool_rag.get_api_documents(content["query"], content["q"], "get_symbol_1")
    if len(response[0]) == 0: # index [0] for document content
        url = f"https://finnhub.io/api/v1/search?q={content['q']}&exchange={content['exchange']}&token={FINNHUB_API_KEY}"
        try:
            response = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            matches = []
            max_failed_match = len(response["result"]) if not by_name else 3
            p_desc_match = 0.80
            p_symb_match = 0.95
            if response["count"] > 0:
                for result in tqdm(response["result"], desc="Score similarity to query"):
                    if max_failed_match > 0:
                        desc = [content['q'].upper(), result["description"].split("-", -1)[0]]
                        symb = [content['q'].upper(), result["symbol"]]
                        if by_name and get_similarity_score(desc) > p_desc_match: 
                            matches.append(result["symbol"])
                        elif not by_name and get_similarity_score(symb) > p_symb_match:
                            matches.append(result["description"])
                            max_failed_match = 0
                        else:
                            max_failed_match -= 1
            if len(matches) > 0:
                tool_rag.add_api_document(content["query"], matches, content["q"], "get_symbol_1")
                return matches
            else:
                return "I don't know."
    else:
        doc = ast.literal_eval(response[0][0])[0]
        return doc["answer"]

def impl_get_name_1(content):
    return impl_get_symbol_1(content, by_name = False)

def impl_get_quote_1(content):
    quotes = tool_rag.get_api_documents(content["query"], content["symbol"], "get_quote_1")
    isOpen = dict(impl_get_market_status_1(content))["isOpen"]
    if len(quotes[0]) == 0 or isOpen: 
        return get_current_price_1(content)
    else:
        last_close = parse(rag_last_market_close(content)).timestamp()
        for quote in quotes[2]: # index [2] for metadata
            if quote["timestamp"] >= last_close:
                return quotes
        return get_current_price_1(content)

def get_current_price_1(content):
    url = f"https://finnhub.io/api/v1/quote?symbol={content['symbol']}&token={FINNHUB_API_KEY}"
    # This is a high-demand endpoint. Expect random failure under heavy (free) use.
    try:
        response = json.loads(requests.get(url).text)
    except:
        return "I don't know."
    else:
        if len(response) > 0 and response["t"] > 0:
            tool_rag.add_quote_document(content["query"], response, content["symbol"], response["t"], "get_quote_1")
            return list(response.items())
        return "I don't know."

def impl_get_market_status_1(content):
    url = f"https://finnhub.io/api/v1/stock/market-status?exchange={content['exchange']}&token={FINNHUB_API_KEY}"
    try:
        response = json.loads(requests.get(url).text)
    except:
        return "I don't know."
    else:
        if len(response) > 0:
            return list(response.items())
        return "I don't know."

def impl_get_peers_1(content):
    docs = tool_rag.get_peers_document(content["query"], content["symbol"], content['grouping'])
    if len(docs[0]) == 0: # index [0] for document content
        url = f"https://finnhub.io/api/v1/stock/peers?symbol={content['symbol']}&grouping={content['grouping']}&token={FINNHUB_API_KEY}"
        try:
            peers = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            if len(peers) > 0:
                names = []
                for peer in peers:
                    if peer == content["symbol"]:
                        continue # skip including the query symbol in peers (included in metadata anyway)
                    name_lookup = dict(q=peer, exchange=content["exchange"], query=content["query"])
                    name = impl_get_name_1(name_lookup)
                    if name != "I don't know.":
                        p = {"symbol": peer, "name": name}
                        names.append(p)
                peers = {"symbol": content["symbol"], "peers": names}
                tool_rag.add_peers_document(content["query"], peers, content["symbol"], "get_peers_1", content['grouping'])
                return list(peers.items())
            return "I don't know."
    else:
        peers = ast.literal_eval(docs[0][0])[0]["answer"] # The first document should be most relevant.
        return list(peers.items())

def impl_local_datetime_1(content):
    local_t = []
    for timestamp in content["t"]:
        local_t.append(local_date_from_epoch(timestamp))
    return local_t

def local_date_from_epoch(timestamp):
    est = pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
    if len(str(timestamp)) == 13:
        return datetime.fromtimestamp(timestamp/1000, tz=est).strftime('%c')
    else:
        return datetime.fromtimestamp(timestamp, tz=est).strftime('%c')

def impl_get_financials_1(content):
    fins = tool_rag.get_api_documents(content["query"], content["symbol"], "get_financials_1")
    if len(fins[0]) == 0:
        url = f"https://finnhub.io/api/v1/stock/metric?symbol={content['symbol']}&metric={content['metric']}&token={FINNHUB_API_KEY}"
        try:
            fin = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            if not fin:
                return "I don't know."
            tool_rag.add_api_document(content["query"], fin, content["symbol"], "get_financials_1")
            return list(fin.items())
    return fins

def impl_get_news_1(content):
    #news = tool_rag.get_api_documents(content["query"], content["symbol"], "get_news_1")
    #if len(news[0]) == 0:
        url = f"https://finnhub.io/api/v1/company-news?symbol={content['symbol']}&from={content['from']}&to={content['to']}&token={FINNHUB_API_KEY}"
        try:
            news = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            if len(news) == 0:
                return "I don't know."
            #tool_rag.add_api_document(content["query"], news, content["symbol"], "get_news_1")
            return list(news.items())
    #return news

def impl_daily_candle_2(content):
    daily_candle = tool_rag.get_api_documents(content["query"], content["stocksTicker"], "daily_candle_2")
    if len(daily_candle[0]) == 0:
        url = f"https://api.polygon.io/v1/open-close/{content['stocksTicker']}/{content['date']}?adjusted={content['adjusted']}&apiKey={POLYGON_API_KEY}"
        try:
            request = requests.get(url)
            daily_candle = ast.literal_eval(request.text)
        except:
            return f"I don't know. Endpoint returned status {request.status_code}"
        else:
            if daily_candle["status"] in ["OK","DELAYED"]:
                tool_rag.add_api_document(content["query"], daily_candle, content["stocksTicker"], "daily_candle_2")
                return list(daily_candle.items())
            else:
                date = parse(content["date"])
                new_date = None
                if date.weekday() == 4: # index 4 for friday
                    new_date = date - timedelta(days=1)
                elif date.weekday() == 0: # index 0 for monday
                    new_date = date - timedelta(days=3)
                if new_date is None:
                    return "I don't know."
                content["date"] = new_date.strftime("%Y-%m-%d")
                return impl_daily_candle_2(content)
    return daily_candle

def impl_custom_candle_2(content):
    url = f"""https://api.polygon.io/v2/aggs/ticker/{content['stocksTicker']}/range/{content['multiplier']}/{content['timespan']}/{content['from']}/{content['to']}?adjusted={content['adjusted']}&sort={content['sort']}&limit={content['limit']}&apiKey={POLYGON_API_KEY}"""
    try:
        request = requests.get(url)
        custom_candle = json.loads(request.text)
    except:
        return f"I don't know. Endpoint returned status {request.status_code}"
    else:
        if custom_candle["status"] in ["OK","DELAYED"]:
            tool_rag.add_api_document(content["query"], custom_candle, content["stocksTicker"], "custom_candle_2")
            return list(custom_candle.items())
        return "I don't know."

def impl_ticker_overview_2(content):
    overview = tool_rag.get_api_documents(content["query"], content["ticker"], "ticker_overview_2")
    if len(overview[0]) == 0:
        url = f"https://api.polygon.io/v3/reference/tickers/{content['ticker']}?apiKey={POLYGON_API_KEY}"
        try:
            request = requests.get(url)
            overview = json.loads(request.text)
        except:
            return f"I don't know. Endpoint returned status {request.status_code}"
        else:
            if overview["status"] in ["OK","DELAYED"]:
                tool_rag.add_api_document(content["query"], overview, content["ticker"], "ticker_overview_2")
                return list(overview.items())
            return "I don't know."
    return overview

def impl_trends_1(content):
    trends = tool_rag.get_api_documents(content["query"], content["symbol"], "trends_1")
    if len(trends[0]) == 0:
        url = f"https://finnhub.io/api/v1/stock/recommendation?symbol={content['symbol']}&token={FINNHUB_API_KEY}"
        try:
            trends = json.loads(requests.get(url).text)
        except:
            return "I don't know."
        else:
            if len(trends) > 0:
                tool_rag.add_api_document(content["query"], trends, content["symbol"], "trends_1")
                return trends
            return "I don't know."
    return trends

def impl_get_news_2(content):
    #news = tool_rag.get_api_documents(content["query"], content["ticker"], "get_news_2")
    #if len(news[0]) == 0:
        url = f"https://api.polygon.io/v2/reference/news?ticker={content['ticker']}&order={content['order']}&limit={content['limit']}&sort={content['sort']}&apiKey={POLYGON_API_KEY}"
        try:
            request = requests.get(url)
            news = json.loads(request.text)
        except:
            return f"I don't know. Endpoint returned status {request.status_code}"
        else:
            if news["status"] in ["OK","DELAYED"]:
                #tool_rag.add_api_document(content["query"], news, content["ticker"], "get_news_2")
                return list(news.items())
            return "I don't know."
    #return news
        
finance_tool = types.Tool(
    function_declarations=[
        get_symbol_1,
        get_name_1,
        get_symbol_quote_1,
        get_market_status_1,
        get_company_peers_1,
        get_local_datetime_1,
        get_last_market_close,
        get_exchange_codes_1,
        get_exchange_code_1,
        get_financials_1,
        get_daily_candlestick_2,
        get_custom_candlestick_2,
        get_ticker_overview_2,
        get_recommendation_trends_1,
        get_news_with_sentiment_2,
        get_rag_tool_response,
        get_wiki_tool_response,
        get_search_tool_response
    ]
)

function_handler = {
    "get_symbol_1": impl_get_symbol_1,
    "get_name_1": impl_get_name_1,
    "get_symbol_quote_1": impl_get_quote_1,
    "get_market_status_1": impl_get_market_status_1,
    "get_company_peers_1": impl_get_peers_1,
    "get_local_datetime_1": impl_local_datetime_1,
    "get_last_market_close": rag_last_market_close,
    "get_exchange_codes_1": rag_exchange_codes_1,
    "get_exchange_code_1": rag_exchange_code_1,
    "get_financials_1": impl_get_financials_1,
    "get_daily_candlestick_2": impl_daily_candle_2,
    "get_custom_candlestick_2": impl_custom_candle_2,
    "get_ticker_overview_2": impl_ticker_overview_2,
    "get_recommendation_trends_1": impl_trends_1,
    "get_news_with_sentiment_2": impl_get_news_2,
    "get_rag_tool_response": ask_rag_tool,
    "get_wiki_tool_response": ask_wiki_tool,
    "get_search_tool_response": ask_search_tool
}

In [19]:
# Define the system prompt.

instruction = f"""You are a helpful and informative bot that answers finance and stock market questions. 
Only answer the question asked and do not change topic. While the answer is still
unknown you must follow these rules for predicting function call order:

RULE#1: Always consult your other functions before get_search_tool_response.
RULE#2: Always consult get_wiki_tool_response before get_search_tool_response.
RULE#3: Always consult get_search_tool_response last.
RULE#4: Always convert timestamps with get_local_datetime_1 and use the converted date/time in your response.
RULE#5: Always incorporate as much useful information from tools and functions in your response."""

In [20]:
# Import the finance api secret keys.

POLYGON_API_KEY = UserSecretsClient().get_secret("POLYGON_API_KEY")
FINNHUB_API_KEY = UserSecretsClient().get_secret("FINNHUB_API_KEY")

In [21]:
# Implement the function calling expert.

@retry.Retry(
    predicate=is_retriable,
    initial=2.0,
    maximum=64.0,
    multiplier=2.0,
    timeout=600,
)
def send_message(prompt):
    #display(Markdown("#### Prompt"))
    #print(prompt, "\n")
    # Define the user prompt part.
    contents = [types.Content(role="user", parts=[types.Part(text=prompt)])]
    # Gemini's innate notion of current date and time is unstable.
    est = pytz.timezone('US/Eastern') # The finance api data is in eastern time.
    contents += f"""
    The current date and time is: {datetime.now(est).strftime('%c')}
    
    Give a concise, and detailed summary. Use information that you learn from the API responses.
    Use your tools and function calls according to the rules. Convert any all-upper case identifiers
    to proper case in your response. Convert any abbreviated or shortened identifiers to their full forms.
    Convert timestamps according to the rules before including them.
    """
    # Enable system prompt, function calling and minimum-randomness.
    config_fncall = types.GenerateContentConfig(
        system_instruction=instruction,
        tools=[finance_tool],
        temperature=0.0
    )
    # Handle cases with multiple chained function calls.
    function_calling_in_process = True
    while function_calling_in_process:
        # Send the user prompt and function declarations.
        response = api.retriable(api.client.models.generate_content, 
                                 model=api(Gemini.Model.GEN), 
                                 config=config_fncall, 
                                 contents=contents)
        # A part can be a function call or natural language response.
        for part in response.candidates[0].content.parts:
            if function_call := part.function_call:
                # Extract the function call.
                fn_name = function_call.name
                #display(Markdown("#### Predicted function name"))
                #print(fn_name, "\n")
                # Extract the function call arguments.
                fn_args = {key: value for key, value in function_call.args.items()}
                #display(Markdown("#### Predicted function arguments"))
                #print(fn_args, "\n")
                # Call the predicted function.
                api_response = function_handler[fn_name](fn_args)[:20000] # Stay within the input token limit
                #display(Markdown("#### API response"))
                #print(api_response[:500], "...", "\n")
                # Create an API response part.
                api_response_part = types.Part.from_function_response(
                    name=fn_name,
                    response={"content": api_response},
                )
                # Append the model's function call part.
                contents.append(types.Content(role="model", parts=[types.Part(function_call=function_call)])) 
                # Append the api response part.
                contents.append(types.Content(role="user", parts=[api_response_part]))
            else:
                # The model gave a natural language response
                function_calling_in_process = False
                break # No more parts in response.
        if not function_calling_in_process:
            break # The function calling chain is complete.
            
    # Show the final natural language summary
    display(Markdown("#### Natural language response"))
    display(Markdown(response.text.replace("$", "\\\\$")))

# Ask a question

In [22]:
send_message("What is the current price of Amazon stock?")

Generate quote embedding: 0it [00:00, ?it/s]


#### Natural language response

The current price of Amazon (AMZN) is \\$186.35. The price changed by -\\$3.63, which is -1.9107% since the previous close. The high price of the day was \\$188.18, and the low was \\$185.53. The opening price for the day was \\$186.51. The previous close price was \\$189.98. The price was last updated on Mon May 5 16:00:00 2025.


In [23]:
send_message(
    """Tell me Amazon's current share price and provide candlestick data for the past month.
    Sort the data in descending order by date. Format the prices consistently as currency.
    Round prices to two decimal places.
    Present the data with multiple columns for display in markdown.""")

Generate quote embedding: 0it [00:00, ?it/s]
Generate api embedding: 0it [00:00, ?it/s]


#### Natural language response

Here is a summary of Amazon's (AMZN) stock information:

As of May 5, 2025, at 16:00:00 local time, the current share price is \\$186.35. This represents a decrease of \\$3.63, or 1.91%, from the previous close.

Here is the candlestick data for the past month, sorted in descending order by date:

| Date               | Open    | High    | Low     | Close   | Volume    |
| ------------------ | ------- | ------- | ------- | ------- | --------- |
| Mon May  5 00:00:00 2025 | \\$186.51 | \\$188.18 | \\$185.53 | \\$186.35 | 35217469  |
| Fri May  2 00:00:00 2025 | \\$191.44 | \\$192.88 | \\$186.40 | \\$189.98 | 77677487  |
| Thu May  1 00:00:00 2025 | \\$190.63 | \\$191.81 | \\$187.50 | \\$190.20 | 74228963  |
| Wed Apr 30 00:00:00 2025 | \\$182.17 | \\$185.05 | \\$178.85 | \\$184.42 | 55176543  |
| Tue Apr 29 00:00:00 2025 | \\$183.99 | \\$188.02 | \\$183.68 | \\$187.39 | 41667255  |
| Mon Apr 28 00:00:00 2025 | \\$190.11 | \\$190.22 | \\$184.89 | \\$187.70 | 33224732  |
| Fri Apr 25 00:00:00 2025 | \\$187.62 | \\$189.94 | \\$185.49 | \\$188.99 | 36413330  |
| Thu Apr 24 00:00:00 2025 | \\$180.92 | \\$186.74 | \\$180.18 | \\$186.54 | 43051696  |
| Wed Apr 23 00:00:00 2025 | \\$183.45 | \\$187.38 | \\$180.19 | \\$180.60 | 63470094  |
| Tue Apr 22 00:00:00 2025 | \\$169.85 | \\$176.78 | \\$169.35 | \\$173.18 | 56607202  |
| Mon Apr 21 00:00:00 2025 | \\$169.60 | \\$169.60 | \\$165.29 | \\$167.32 | 48126111  |
| Thu Apr 17 00:00:00 2025 | \\$176.00 | \\$176.21 | \\$172.00 | \\$172.61 | 44726453  |
| Wed Apr 16 00:00:00 2025 | \\$176.29 | \\$179.10 | \\$171.41 | \\$174.33 | 51866916  |
| Tue Apr 15 00:00:00 2025 | \\$181.41 | \\$182.35 | \\$177.93 | \\$179.59 | 43617902  |
| Mon Apr 14 00:00:00 2025 | \\$186.84 | \\$187.44 | \\$179.23 | \\$182.12 | 48002540  |
| Fri Apr 11 00:00:00 2025 | \\$179.93 | \\$185.86 | \\$178.00 | \\$184.87 | 50594339  |
| Thu Apr 10 00:00:00 2025 | \\$185.44 | \\$186.87 | \\$175.85 | \\$181.22 | 68302045  |
| Wed Apr  9 00:00:00 2025 | \\$172.12 | \\$192.65 | \\$169.93 | \\$191.10 | 116804328 |
| Tue Apr  8 00:00:00 2025 | \\$185.23 | \\$185.90 | \\$168.57 | \\$170.66 | 87710360  |
| Mon Apr  7 00:00:00 2025 | \\$162.00 | \\$183.41 | \\$161.38 | \\$175.26 | 109297115 |


In [24]:
send_message(
    '''Tell me about Amazon's current bullish versus bearish predictions, and recommendation trends.
    Include a discussion of any short-term trends, and sentiment analysis.''')

Generate api embedding: 0it [00:00, ?it/s]


#### Natural language response

As of May 6, 2025, here's a summary of Amazon's bullish versus bearish predictions and recommendation trends:

**Recommendation Trends:**

*   Based on analyst recommendations, there's a strong bullish sentiment towards Amazon.
*   In May 2025, there were 22 strong buy, 51 buy, and 6 hold recommendations. There were no sell or strong sell recommendations.
*   The trend has been consistent over the past few months, with a high number of buy and strong buy recommendations.

**Sentiment Analysis:**

*   Recent news articles present a mixed sentiment regarding Amazon.
*   Some articles highlight Amazon's strong financial performance, growth in Amazon Web Services (AWS), and potential in artificial intelligence (AI).
*   Other articles express concerns about tariffs, a weakening economy, and increasing competition in the cloud computing market.
*   Several articles suggest that Amazon is well-positioned for long-term growth due to its diversified business model and investments in AI.
*   Some articles indicate that Amazon's stock has underperformed compared to some of its peers in 2025.
*   Recent news suggests that Amazon debunked a rumor that hurt AI-related stocks.

**Short-Term Trends:**

*   There are concerns about the impact of tariffs on Amazon's e-commerce business.
*   Some analysts believe that Amazon's cloud business and AI investments can offset the negative effects of tariffs.
*   The stock market may see significant volatility this week due to key earnings reports from major tech companies, including Amazon.

In summary, while there are some short-term concerns related to tariffs and the economy, the overall sentiment towards Amazon remains positive, with a strong consensus among analysts that the company is well-positioned for long-term growth.


In [25]:
send_message("What is Google's stock ticker symbol?")

Score similarity to query: 100%|██████████| 11/11 [00:00<00:00, 20.72it/s]
Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]


#### Natural language response

Google's stock ticker symbols on the NASDAQ are GOOGL and GOOG. It is also listed on the Frankfurt Stock Exchange under the ticker symbol GGQ1. These ticker symbols now refer to Alphabet Inc., Google's holding company.


In [29]:
send_message(
    '''Tell me about Google's share price over the past month.
    Perform a sentiment analysis of news during the same period. Include trends.''')

Score similarity to query: 100%|██████████| 11/11 [00:00<00:00, 20.75it/s]
Generate api embedding: 0it [00:00, ?it/s]


#### Natural language response

Here's a summary of Google's (Alphabet Inc.) stock performance and news sentiment over the past month (April 6, 2025, to May 5, 2025):

**Stock Performance:**

*   Over the past month, Google's stock (GOOGL) has experienced price fluctuations.
*   The lowest price was \\$140.53 on April 7, 2025.
*   The highest price was \\$166.10 on April 25, 2025.
*   The stock closed at \\$164.21 on May 5, 2025.

**News Sentiment:**

The news sentiment surrounding Google (GOOGL) over the past month has been mixed, with a slight positive trend overall. Key themes include:

*   **Artificial Intelligence (AI):** Google is heavily involved in AI, with both positive and negative implications.
    *   Positive: Google is recognized as a leader in AI, with its Google Cloud and Gemini AI model positioning it to compete with rivals. Its AI-powered search features are seeing increasing usage.
    *   Neutral: Google is competing in the quantum computing field.
    *   Negative: Google faces competition from other AI upstarts.
*   **Financial Performance:**
    *   Positive: Alphabet reported strong Q1 2025 earnings, with revenue, operating income, and earnings per share growth. The company also announced a \\\$70 billion share buyback and increased its dividend.
    *   Negative: There are concerns over increased spending and the sustainability of Google Search.
*   **Partnerships and Collaborations:**
    *   Positive: Google Cloud is collaborating with Clover Health to enhance AI-powered search capabilities for clinicians.
    *   Neutral: There were reports of Google being in talks to rent AI servers from CoreWeave.
*   **Competition:**
    *   Negative: Amazon is outgrowing Google in cloud computing and AI.
*   **Legal and Regulatory Issues:**
    *   Negative: Google faces antitrust lawsuits from the U.S. Justice Department, alleging the company has illegal monopolies in online search and ad technology software.
*   **General Market Trends:**
    *   Negative: The U.S. economy contracted in the first quarter of 2025, raising fears of a potential recession, which negatively impacted tech giants like Google.

In summary, while Google's stock price has fluctuated over the past month, the news sentiment has been generally positive, driven by the company's strong position in AI and its solid financial performance. However, investors should be aware of the potential risks from competition, legal and regulatory issues, and macroeconomic uncertainty.


In [30]:
send_message("What is MGM Studio's stock symbol?")

Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]


#### Natural language response

I was unable to find the stock symbol for Mgm Studio.

In [31]:
send_message("What is Amazon MGM Studio's stock symbol?")

Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]
Generate grounding embedding: 0it [00:00, ?it/s]


#### Natural language response

Amazon Mgm Studios is a subsidiary of Amazon, and its stock is traded on the NASDAQ under the symbol AMZN.


In [32]:
send_message("What is Facebook's stock ticker symbol?")

Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]
Generate grounding embedding: 0it [00:00, ?it/s]


#### Natural language response

Facebook's stock ticker symbol is META. It is listed on the NASDAQ. Facebook's initial ticker symbol was FB, but it changed to META on June 9.


In [33]:
send_message(
    '''How is the outlook for Apple based on trends and news sentiment over the past month?
    Perform the same analysis on Apple's peers. Then compare Apple result to it's peers.''')

Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 56.15it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 71.11it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 67.94it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 17/17 [00:00<00:00, 69.64it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 66.33it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 55.61it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 18/18 [00:00<00:00, 51.38it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 17/17 [00:00<00:00, 47.80it/s]
Generate api embedding: 0it [00:00, ?it/s]
Generate peer

#### Natural language response

Based on the available data, here's a summary of the outlook for Apple and its peers:

**Apple (AAPL):**

*   **Analyst Recommendations:** Analyst recommendations for Apple have remained relatively stable over the past few months, with a majority of analysts rating the stock as a "buy" or "strong buy." In May 2025, there were 39 buy or strong buy recommendations, 14 hold recommendations, and 3 sell recommendations.
*   **News Sentiment:** Recent news sentiment for Apple is mixed. Some articles highlight the company's strong Q1 earnings and potential for growth in AI, while others express concerns about tariffs, weaker sales in China, and regulatory challenges. There are also reports of Warren Buffett reducing Berkshire Hathaway's stake in Apple.
*   **Tariffs:** Tariffs continue to be a headwind for Apple, with the company expecting a \\$900 million increase in tariff-related costs in Q3. However, Apple has taken steps to mitigate the impact by shifting production to countries like India and Vietnam.
*   **AI:** Apple is facing pressure to roll out AI features for future iPhones, and there are concerns about the company's innovation and revenue growth. However, Apple is also exploring partnerships with companies like Broadcom to develop AI chips.
*   **Financial Services:** Apple's financial services business, including Apple Pay and Apple Card, is experiencing success and attracting interest from major financial institutions.
*   **China:** Weaker iPhone sales in China are a concern for Apple, and the company faces an uncertain outlook due to trade tensions and weaker sales in the region.

**Apple's Peers:**

Based on the API, Apple's peers in the subIndustry are: Dell Technologies, HP Inc, Hewlett Packard Enterprise, NetApp Inc, Super Micro Computer Inc, Western Digital Corp, Pure Storage Inc - Class A, and IonQ Inc.

*   **Dell Technologies (DELL):** Dell is well-positioned to benefit from the AI surge with its expanded portfolio and Dell AI Factory. The company has consistently beaten earnings estimates, and its current stock price aligns with the bottom price target, making it an optimal entry point. Dell is also a key player in the IoT Analytics market, and is expected to benefit from the growth of the Open RAN market.
*   **HP Inc (HPQ):** HP is also expected to benefit from the tariff relief. The company is a key player in the Chromebook market.
*   **Super Micro Computer Inc (SMCI):** Super Micro Computer is facing challenges such as declining gross margins, allegations from a short seller, and a potential DOJ probe.

**Comparison:**

*   Apple faces headwinds from tariffs and slowing innovation, while peers like Dell are well-positioned to benefit from the AI surge.
*   Analyst recommendations for Apple are generally positive, but news sentiment is mixed, with concerns about tariffs and competition.
*   Dell is seen as a strong player in the AI server market, with a diversified portfolio and attractive valuation.

**Overall Summary:**

The outlook for Apple is mixed, with both opportunities and challenges. While the company has a strong brand and a large installed base, it faces headwinds from tariffs, competition, and slowing innovation. Peers like Dell are well-positioned to benefit from the AI surge, but also face their own challenges. Investors should carefully consider these factors when making investment decisions.

Recommendation trends for Apple are still mostly positive, but the news sentiment suggests caution due to potential headwinds.


In [34]:
send_message(
    '''What does the recent news say about Apple and the impact of tariffs? Over the past 2 month.
    Avoid discussing duplicate news entries.''')

#### Natural language response

Recent news over the past two months regarding Apple and the impact of tariffs indicates a complex and evolving situation. Here's a summary:

**Tariff Impact and Mitigation:**

*   **Earnings Impact:** Tariffs have impacted Apple's earnings, and the company expects a \\\$900 million increase in tariff-related costs in the third quarter of the fiscal year.
*   **Mitigation Strategies:** Apple has been taking steps to mitigate the impact of tariffs, including building up inventory and shifting production to countries like India and Vietnam to avoid higher tariffs on Chinese imports.
*   **Production Shifts:** Apple is shifting more iPhone production to India to mitigate the impact of tariffs.

**Uncertainty and Challenges:**

*   **Uncertain Outlook:** Apple faces an uncertain outlook due to tariff concerns and weaker sales in China.
*   **Headwinds:** Apple faces multiple challenges, including regulatory issues, tariff-related cost increases, and supply chain issues, leading to concerns about its near-term growth prospects.
*   **Trade Tensions:** Apple is at the center of increasing geopolitical tensions between the United States and China.

**Analyst Perspectives:**

*   **Price Target Cuts:** Bank of America cut Apple's price target due to tariff risks and delayed artificial intelligence rollout for future iPhones.
*   **Positive Outlook:** Despite the challenges, some analysts remain bullish on Apple, citing the company's strong market share in the premium hardware sector and its flywheel model.

**Financial Performance and Strategies:**

*   **Solid Results:** Apple reported better-than-expected second-quarter results, but faces an uncertain outlook due to tariff concerns and weaker sales in China.
*   **Cash Generation:** Apple remains a cash-generation machine, returning more to shareholders through dividends and buybacks.
*   **Stock Buybacks:** Apple has been a leader in share buybacks, repurchasing \\\$695 billion of stock in the past decade.

**Overall Sentiment:**

The news sentiment surrounding Apple and tariffs is mixed. While the company has taken steps to mitigate the impact of tariffs and continues to generate strong cash flow, it faces significant challenges and uncertainty due to ongoing trade tensions and potential cost increases.


# Conclusion

<span style="font-size:18px;">
For now that will have to do. Our Essy has a solid foundation but more could be done to organise metadata. No evaluation or validation has been performed (except fuzzing the prompt). Next steps include restructuring the vector database based on lessons learned. That'll be followed by plotting, multi-modal, and structured output. The last close date (generative) function can be temperamental. In the same way Gemini always feels regarding dates. I've learnt so much. I'm happy I decided to participate in the event! It really has been a joy to see Essy grow from random chat with Gemini into the foundation for a good-broker buddy. I hope you enjoy playing with this edition as much as I enjoyed building it!
</span>